In [98]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("preprocesamiento").getOrCreate()

# Cargamos el dataset
df_anime = spark.read.csv("anime.csv", header=True, inferSchema=True, encoding="UTF-8")
df_anime.printSchema()
df = spark.read.option("delimiter", ",").option("quote", '"').option("escape", '"').csv("anime.csv", header=True, inferSchema=True)


+-------+
|Type   |
+-------+
|TV     |
|Special|
|Unknown|
|OVA    |
|Music  |
|Movie  |
|ONA    |
+-------+



In [11]:
df_anime.show()

+---+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+----------------+-----------+---------------+--------------------+------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+
| ID|                Name|Score|              Genres|        English name|                 Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|         Studios|     Source|       Duration|              Rating|Ranked|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7| Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+---+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+------------

In [63]:
from pyspark.sql.functions import regexp_replace

In [78]:
#df_anime = df_anime.withColumn("Name", regexp_replace("Name", "[\'\"]+", "'"))
df_anime = df_anime.withColumn("Name", regexp_replace("Name", '''["']{2,}''', "'"))

In [68]:
filtered_df = df_anime.filter(df_anime["Type"] != "OVA")

AttributeError: 'NoneType' object has no attribute 'filter'

In [90]:
df_anime.select("Type").distinct().show(20, False)

+-------------------------------------------------------------+
|Type                                                         |
+-------------------------------------------------------------+
|TV                                                           |
|マリッジブルー 「婚約者がいるのに, どうしてこんな男に......」|
|Special                                                      |
|Unknown                                                      |
|逆転裁判 ～その「真実」、異議あり！～                        |
|OVA                                                          |
|"""いそっぷ""のおはなしより 牛とかえる                       |
|Music                                                        |
|Movie                                                        |
|ONA                                                          |
|映画演説 政治の倫理化 後藤新平 1926                          |
|逆転裁判 ～その「真実」、異議あり！～ Season 2               |
+-------------------------------------------------------------+



In [100]:
df.where(df["ID"] == "37490").show(20, False)

+-----+----------------------------------------------------+-----+------------------------------+---------------------+----------------------------------------------+----+--------+---------------------------+---------+---------------+----------+-----------+------+---------------+-------------------------+------+----------+-------+---------+--------+---------+-------+-------+-------------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|ID   |Name                                                |Score|Genres                        |English name         |Japanese name                                 |Type|Episodes|Aired                      |Premiered|Producers      |Licensors |Studios    |Source|Duration       |Rating                   |Ranked|Popularity|Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10|Score-9|Score-8|Score-7|Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+-----+-----------------------------------------------

In [32]:
filtered_df.show()

+---+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+--------------------+-----------+--------------------+--------------------+----------------+-----------+---------------+--------------------+------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+
| ID|                Name|Score|              Genres|        English name|                 Japanese name| Type|Episodes|               Aired|  Premiered|           Producers|           Licensors|         Studios|     Source|       Duration|              Rating|Ranked|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7| Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+---+--------------------+-----+--------------------+--------------------+------------------------------+-----+--------+------------

0

In [45]:
filtered_df = filtered_df.drop("English name","Japanese name", "Premiered", "Aired", "Licensors")
filtered_df.show()

+---+--------------------+-----+--------------------+-----+--------+--------------------+----------------+-----------+---------------+--------------------+------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+
| ID|                Name|Score|              Genres| Type|Episodes|           Producers|         Studios|     Source|       Duration|              Rating|Ranked|Popularity|  Members|Favorites|Watching|Completed|On-Hold|Dropped|Plan to Watch|Score-10| Score-9| Score-8| Score-7| Score-6|Score-5|Score-4|Score-3|Score-2|Score-1|
+---+--------------------+-----+--------------------+-----+--------+--------------------+----------------+-----------+---------------+--------------------+------+----------+---------+---------+--------+---------+-------+-------+-------------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+
|  1|        Cow

In [34]:
from pyspark.sql.functions import col

# Assuming df is your Spark DataFrame
columns_with_unknown = []

for col_name in df_anime.columns:
    # Checking for "unknown" value in each column
    count_unknown = df_anime.filter(col(col_name) == "Unknown").count()
    if count_unknown > 0:
        columns_with_unknown.append(col_name)

print("Columns with 'unknown' values:", columns_with_unknown)

Columns with 'unknown' values: ['Score', 'Genres', 'English name', 'Japanese name', 'Type', 'Episodes', 'Aired', 'Premiered', 'Producers', 'Licensors', 'Studios', 'Source', 'Duration', 'Rating', 'Ranked', 'Popularity', 'Score-10', 'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4', 'Score-3', 'Score-2', 'Score-1']


In [42]:
df_anime.where(df_anime['Score'] == "Unknown").count()

5140

In [80]:
import pandas as pd

pandas_anime = pd.read_csv("anime.csv")
pandas_anime.head()

,ID,Name,Score,Genres,English name,Japanese name,Type,Episodes,Aired,Premiered,...,Score-10,Score-9,Score-8,Score-7,Score-6,Score-5,Score-4,Score-3,Score-2,Score-1
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space",Cowboy Bebop,カウボーイビバップ,TV,26,"Apr 3, 1998 to Apr 24, 1999",Spring 1998,...,229170.0,182126.0,131625.0,62330.0,20688.0,8904.0,3184.0,1357.0,741.0,1580.0
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space",Cowboy Bebop:The Movie,カウボーイビバップ 天国の扉,Movie,1,"Sep 1, 2001",Unknown,...,30043.0,49201.0,49505.0,22632.0,5805.0,1877.0,577.0,221.0,109.0,379.0
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen",Trigun,トライガン,TV,26,"Apr 1, 1998 to Sep 30, 1998",Spring 1998,...,50229.0,75651.0,86142.0,49432.0,15376.0,5838.0,1965.0,664.0,316.0,533.0
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),TV,26,"Jul 2, 2002 to Dec 24, 2002",Summer 2002,...,2182.0,4806.0,10128.0,11618.0,5709.0,2920.0,1083.0,353.0,164.0,131.0
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",Beet the Vandel Buster,冒険王ビィト,TV,52,"Sep 30, 2004 to Sep 29, 2005",Fall 2004,...,312.0,529.0,1242.0,1713.0,1068.0,634.0,265.0,83.0,50.0,27.0


In [92]:
pandas_anime.

AttributeError: 'DataFrame' object has no attribute 'encoding'